### Setup

In [ ]:
import os
import pandas as pd
from datetime import date
from oil_optimization.utils.io_helpers import read_yaml
from datetime import timedelta
import plotly.io as pio
import plotly.express as px
import matplotlib.pyplot as plt

def setup():
    config = read_yaml('config/config.yml')
    data_dir = config['data_ingestion']['root_dir']
    raw_data_dir = data_dir + '/raw'
    file_list = os.listdir(raw_data_dir)

    dataframes = {}
    for filename in file_list:
        key = filename[:-4] #Removing .csv extension from filename
        try:
            dataframes[key] = pd.read_csv(f'{raw_data_dir}/{filename}', parse_dates=['period'])
        except FileNotFoundError as e:
            print(e)
            
    return dataframes   

# os.chdir('..')
print(os.getcwd())
df_dict = setup()
pd.set_option('plotting.backend','plotly')
pio.templates.default = "plotly_dark"

c:\Users\User\Desktop\DS\Projects\oil-optimization


In [6]:
def plot_values(df, label, title):
    fig = df[['period','value']].set_index('period').rename({'value':label},axis=1).plot()
    fig.update_layout(showlegend=False,
                      title=title)
    return fig

### WTI Price

In [15]:
df_wti = df_dict['wti_prices']
plot_values(df_wti,label='wti_prices', title='Daily WTI Price Time Series (Dollars per Barrel)')

In [13]:
df_dict['cpi_energy']

,realtime_start,realtime_end,period,value
0,2025-10-31,2025-10-31,2015-01-01,199.926
1,2025-10-31,2025-10-31,2015-02-01,203.021
2,2025-10-31,2025-10-31,2015-03-01,205.757
3,2025-10-31,2025-10-31,2015-04-01,203.577
4,2025-10-31,2025-10-31,2015-05-01,208.973
...,...,...,...,...
124,2025-10-31,2025-10-31,2025-05-01,275.406
125,2025-10-31,2025-10-31,2025-06-01,278.012
126,2025-10-31,2025-10-31,2025-07-01,275.044
127,2025-10-31,2025-10-31,2025-08-01,276.936


### Cpi Energy

In [16]:
df_cpi = df_dict['cpi_energy']
plot_values(df_cpi, 'cpi_energy','U.S. Energy CPI: Monthly Time Series')

### Gasoline Price

In [7]:
df_gasoline = df_dict['gasoline_price']
df_filter = df_gasoline[['period','area-name','value']]
df_pivot = df_filter.pivot(columns='area-name',values='value',index='period')

In [8]:
states = ['CALIFORNIA','COLORADO','FLORIDA','MASSACHUSETTS','OHIO','FLORIDA','TEXAS','MINNESOTA','NEW YORK','WASHINGTON']
cities = ['BOSTON','LOS ANGELES','SAN FRANCISCO','CHICAGO','CLEVELAND','DENVER','MIAMI','HOUSTON','NEW YORK CITY','SEATTLE']
Padds = [f'PADD {i}' for i in range(1,6)]

In [11]:
import plotly.graph_objects as go
from plotly.colors import n_colors
import numpy as np
import pandas as pd

# 12 sets of normal distributed random data, with increasing mean and standard deviation
def plotting_gasoline_by_group(df, groups, group_name):
    colors = n_colors('rgb(5, 200, 200)', 'rgb(200, 10, 10)', len(groups), colortype='rgb')

    fig = go.Figure()
    for data_line, color, zone in zip(df[groups].T.to_numpy(), colors, df[groups].columns):
        fig.add_trace(go.Violin(x=data_line, line_color=color, name=zone))

    fig.update_traces(orientation='h', side='positive', width=3, points=False)
    fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=False,title=f'Gasoline Price by {group_name}')
    fig.show()

In [12]:
plotting_gasoline_by_group(df_pivot,states,'States')

In [21]:
plotting_gasoline_by_group(df_pivot,cities,'Cities')

In [23]:
plotting_gasoline_by_group(df_pivot,Padds,'PADDS')

### Imports and Exports

In [13]:
df_imp_exp = df_dict['imports_and_exports']
cols = ['period','process-name','value']
df_imp_exp = df_imp_exp[cols]
df_imp_exp = df_imp_exp[df_imp_exp['process-name'].isin(['Imports','Exports'])]

df_imp_exp = df_imp_exp.pivot(columns=['process-name'],values='value',index='period')

In [16]:
df_imp_exp

process-name,Exports,Imports
period,,
2015-01-02,388,6856
2015-01-09,388,7492
2015-01-16,388,7218
2015-01-23,502,7422
2015-01-30,502,7387
...,...,...
2025-09-26,3751,5833
2025-10-03,3590,6403
2025-10-10,4466,5525


In [14]:
fig = df_imp_exp.groupby([df_imp_exp.index.strftime('%Y-%m')]).mean().plot()
fig.update_layout(title='U.S. Monthly Oil Imports and Exports (Thousand Barrels per Day)',
                  legend=None)

### Input Utilization

In [14]:
df_input = df_dict['input_utilization']


df_input_sorted = df_input.sort_values(by=['period','area-name'])
df_input_refinery = df_input_sorted.pivot(index='period',columns='area-name',values='value')
df_input_refinery.columns = ['East Coast (PADD 1)',
                             'Midwest (PADD 2)',
                             'Gulf Coast (PADD 3)',
                             'Rocky Mountain (PADD 4)',
                             'West Coast (PADD 5)']


In [23]:
df_input['series-description'].unique()

array(['Midwest (PADD 2) Refiner Net Input of Crude Oil (Thousand Barrels per Day)',
       'Rocky Mountain (PADD 4) Refiner Net Input of Crude Oil (Thousand Barrels per Day)',
       'West Coast (PADD 5) Refiner Net Input of Crude Oil (Thousand Barrels per Day)',
       'East Coast (PADD 1) Refiner Net Input of Crude Oil (Thousand Barrels per Day)',
       'Gulf Coast (PADD 3) Refiner Net Input of Crude Oil (Thousand Barrels per Day)'],
      dtype=object)

In [15]:
fig = df_input_refinery.plot()
fig.update_layout(title='Refiner Net Input of Crude Oil per PADD (Thousand Barrels per Day)',
                  legend=None)

<img src='https://www.eia.gov/analysis/transportationfuels/padd5/images/fig1.jpg'>

### Oil Production

In [11]:
df_prod = df_dict['oil_production']

df_prod_filter = df_prod[df_prod['units'] == 'MBBL']
df_prod_sum = df_prod_filter.groupby('period')['value'].sum().reset_index()

df_prod_sum['year_month'] = df_prod_sum['period'].dt.strftime('%Y-%m')
plot_values(df_prod_sum,'oil_production','U.S. Total Field Production (Thousand Barrels per Day)')

### U.S. Dollar Index

In [35]:
df_idx = df_dict['us_dollar_index']
df_idx['value'] = pd.to_numeric(df_idx['value'],errors='coerce')
plot_values(df_idx,'us_dollar_index',title='Daily U.S. Dollar Index Time Series')

### Volatility Index

In [36]:
df_vidx = df_dict['volatility_index']
df_vidx['value'] = pd.to_numeric(df_vidx['value'],errors='coerce')
plot_values(df_vidx,'volatility_index',title='Daily Volatility Index Time Series')